# Notebook showing basic operations of Kernel Memory - Document Ingestion

Prompt for secrets from the user and save in a local file.  Ensure that the local file is not checked
into source control.

In this case, the only secret is the Azure Open AI Key.

In [ ]:
// Work-around for Console.Readline() not working in polyglot notebooks: https://github.com/dotnet/interactive/blob/main/docs/input-prompts.md

using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string consoleInput(string prompt) {
    Task<string> inputTask = InteractiveKernel.GetInputAsync(prompt);
    string input = inputTask.Result;
    return input;
}

//Console.WriteLine($"Hello, {consoleInput("What's your name?")}!");


In [ ]:
using System;

// Global configuration variables
var SECRETS_FILE = "azure_secrets.json";
var AZURE_OPENAI_ENDPOINT = "https://ragchatopenai.openai.azure.com/";
var POSTGRESS_SQL_ENDPOINT = "https://ragchatpostgres.postgres.database.azure.com/";
var OPENAI_MODEL = "gpt-4o";

string[] secretKeys = {"AZURE_OPENAI_KEY", "POSTGRES_SQL_PW"};

Dictionary<string, string> secrets = new Dictionary<string, string>();

if (System.IO.File.Exists(SECRETS_FILE)) {
    secrets = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(System.IO.File.ReadAllText(SECRETS_FILE));
} else {
    Console.WriteLine("Secrets file not found. Please enter the following secrets:");
    foreach (string key in secretKeys) {
        secrets[key] = consoleInput($"Please enter the value for {key}: ");
    }
    System.IO.File.WriteAllText(SECRETS_FILE, System.Text.Json.JsonSerializer.Serialize(secrets));
}

var AZURE_OPENAI_KEY = secrets["AZURE_OPENAI_KEY"];
var POSTGRES_SQL_PW = secrets["POSTGRES_SQL_PW"];


Import the required libraries and add a using statement.

In this case, the only required library is the Kernel Memory system.  Just get the latest stable version of the library (i.e. don't specify a version).

In [ ]:
#r "nuget: Microsoft.KernelMemory.Core"

using Microsoft.KernelMemory;

Instantiate the framework.  Pass in two configurations - an embedding library to create the vector embeddings
using ADA002, and a configuration to ask questions of a LLM.  In this case, use GPT 4Omni.

In [ ]:
var embeddingConfig = new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "Embedding",
    Endpoint = AZURE_OPENAI_ENDPOINT,
    APIKey = AZURE_OPENAI_KEY
};

var completionConfig = new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "GPT4o",
    Endpoint = AZURE_OPENAI_ENDPOINT,
    APIKey = AZURE_OPENAI_KEY
};

var kernelMemory = new KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(config: embeddingConfig)
    .WithAzureOpenAITextGeneration(config: completionConfig)
    .Build();

Ingest a document.  This will create vector embeddings and store them in-memory.  This will store the full document broken up in chunk (snippets), with each chunk indexed by a vector describing the relevant aspects.  Kernel Memory then has the ability to search through the relevant chunks and only send the applicable ones to the LLM when a question is asked.

In [ ]:
var docId = kernelMemory.ImportDocumentAsync(@".\documents\Recipies.docx").Result;
Console.WriteLine(docId);

Ask multiple questions of the indexed memory.  For each question, the memory will retrieve relevant information from the vector store, and then send them to the LLM along with the question.  If no relevant information is found, the answer returned is: "INFO NOT FOUND" (this is configurable).

In [ ]:
string[] questions =
{
    "Give me a pescatarian recipie for Sui Mai dumplings",
    "What's the difference between regular bread and no-knead bread?",
};

foreach (var question in questions)
{
    Console.WriteLine($"Question: {question}");
    var result = kernelMemory.AskAsync(question).Result;
    Console.WriteLine($"Response: {result}");
    Console.WriteLine("-------------------------------------------------------------");
}


Search Memory for relevant results based on the question.  This will return full snippets of the source document which are relevant, along with a score which indicates the relevancy.

In [ ]:
var response = kernelMemory.SearchAsync("How do I make dolmades?").Result;
response.Display();


Create a new instance of Kernel Memory, this time configuring it to use PostgreSQL as the store for vectors.

In [ ]:
var connString = $"Host=salearningpostgres.postgres.database.azure.com;Database=ragchat0;Username=SALearningAdmin;Password={POSTGRES_SQL_PW};SSLMode=Require;Trust Server Certificate=true";

// AI Kernel Memory - Persist vectors to Postgres
var postConfig = new PostgresConfig {
    ConnectionString = connString
};

var Endpoint = "https://ragchatopenai.openai.azure.com/";

var kernelMemoryDB = new KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(config: embeddingConfig)
    .WithAzureOpenAITextGeneration(config: completionConfig)
    .WithPostgresMemoryDb(config: postConfig)
    .Build();

Ingest all files, in Launch directory, this time saving into database.

In [ ]:
var docId = kernelMemoryDB.ImportDocumentAsync(@".\documents\Recipies.docx").Result;
Console.WriteLine(docId);

Ask question, this time using the saved vector embeddings in the database as opposed to the in memory store.

In [ ]:
foreach (var question in questions)
{
    Console.WriteLine($"Question: {question}");
    var result = kernelMemoryDB.AskAsync(question).Result;
    Console.WriteLine($"Response: {result}");
    Console.WriteLine("-------------------------------------------------------------");
}

